In [1]:
from parameter_object import parameterClass
%pylab inline
#Imports .mdf files for providing ground-truth data.
from sklearn.cross_validation import ShuffleSplit
import numpy as np
#sys.path.append('resources')
#sys.path.append('final_software')
import v2_functions as v2
reload(v2)
from scipy.ndimage import measurements, center_of_mass,filters
from sklearn.ensemble import ExtraTreesRegressor
import time
import tifffile as tif_fn
import csv
import numpy as np
import cPickle as pickle
def read_ground_truth(file_name,time_pt):
    with open(str(file_name),'r') as f:
        txt = f.readlines()
    track ={}
    temp_x = []
    temp_y = [];
    temp_z = [];
    name = []
    #Populates new array if line is 'Track', and adds points if labelled 'Point'
    for i in range(0,txt.__len__()):
        line = txt[i].split(' ')
        if line[0] == 'Track':
            name.append(line[1])
            if temp_x.__len__() >0:
                track[int(name[-2])] = [temp_x, temp_y, temp_z]
            temp_x = [];
            temp_y = [];
            temp_z = [];

        elif line[0] == 'Point':        
            temp_x.append(float(line[2]))
            temp_y.append(float(line[3]))
            temp_z.append(float(line[5]))
    
    
    if temp_x.__len__() >0:
        track[int(name[-1])] = [temp_x, temp_y, temp_z]
    
    #Empty array.
    lengths = []
    dots = []
    for t in track:
        lengths.append(track[t][0].__len__())
    
    #Finds the median point on the label.
    average_height = np.median(np.array(lengths))
    center_pt = int((average_height-1)/2)
    
    #Find the rectangles of interest.
    rects = (time_pt, int(0), int(0), int(abs(par_obj.width)), int(abs(par_obj.height)))

    #accepts and calculates track centers from tracks which are at least larger than half the median.
    track_center = []
    for t in track:
        x = track[t][0]
        y = track[t][1]
        z = track[t][2]
        if x.__len__() > center_pt:
            #Always calculates the track center.
            cent_x = x[int(round(x.__len__()/2,0))]
            cent_y = y[int(round(x.__len__()/2,0))]
            cent_z = z[int(round(x.__len__()/2,0))]
        track_center.append([cent_y,cent_x,cent_z])

    return track_center

Populating the interactive namespace from numpy and matplotlib


In [ ]:

#This is a class and function which takes over the print output in the absence of the gui.
class intObject():
    def __init__(self):
        self.pa = None
    def report_progress(self,string):
        d=0
        print(string)
        
    
#Here we choose which of our experiments to run.

for experiment in [1]:
    

    
    if experiment == 1:
        path = '/Users/martin/Pictures/Live_mdf/'
        tif_out_path = '/Users/martin/Pictures/Live_mdf/out'

    #we open the csv file for writing and write header line
    csvPath = path+"out/";
    output= ['TP',
            'FP',
             'FN',
             'time_pt',
             'rel_thr',
             'training',
             'file_no']
    with open(csvPath+'outputData.csv', 'wb') as csvfile:
        spamwriter = csv.writer(csvfile,  dialect='excel')
        spamwriter.writerow(output)

    #This is where we have the parameter sets.
    par_obj = parameterClass()
    par_obj.TP={}
    par_obj.FP={}
    par_obj.FN={}
    for training in range(1,2):
        
        #par_obj.num_of_tree = training
        par_obj.TP[training]={}
        par_obj.FP[training]={}
        par_obj.FN[training]={}

        #Number of times to repeat.
        for bc in range(0,2):
            #reset array of memmaps
            par_obj.tiffarray=[]
            
            int_obj = intObject()
            
            
            ####If we have cross-validation.
            #if experiment==1:
            #    ss = ShuffleSplit(30, n_iter=1, test_size=15, train_size=num_of_train)
            #for train_index, test_index in ss:
                 #print("%s %s" % (train_index, test_index))
            ####
            ###Remove the following in favour of the above cross-validation
            train_index = [0]
            test_index = [0]
            ###
    
            temp_index = list(train_index)
            #temp_index.extend(list(test_index))
            #test_index = temp_index
            print train_index, test_index

            int_obj = intObject()
            par_obj.gt_sum = {}
            par_obj.gt_dense ={}
            par_obj.f_matrix =[]
            par_obj.o_patches=[]
            par_obj.gt_sum = {}
            par_obj.gt_dense ={}
            par_obj.gt_file_array = []
            par_obj.frames_2_load = {}
            par_obj.gt_array =[]
            
            
            #####
            #####
            ####May want to change these of course for the hessian and maxima finding.
            par_obj.z_calibration=3
            par_obj.min_distance=[15,15,3]
            par_obj.abs_thr=0
            par_obj.rel_thr=0
            par_obj.max_features = 52
            par_obj.num_of_tree = 36
            ####
            #####
            ######
            
            par_obj.feature_type = 'fine3'

            if experiment== 1:
                
                nstr = [0]*1#Number of files to upload
            
            
            datasets=['dense_arr','feat_arr','double_feat_arr','pred_arr','sum_pred','maxi_arr','pts','roi_stk_x','roi_stk_y','roi_stkint_x','roi_stkint_y']
            par_obj.data_store={}
            
            for dataname in datasets:
                par_obj.data_store[dataname]={}

            #We initalize each of our images in our experiments.
            for i in range(0,nstr.__len__()):
                
                n = str(i).zfill(3)
                if experiment== 1:
                    par_obj.ch_active =[0,1]
                    par_obj.sigma =  float(15);
                    pixel_cut_off_3D = 15.0
                    #par_obj.limit_time_pt = 10 #########HERE IS TO BE DELETED
                    
                    #i =0 is the first datafile. To add more do i==1 etc.
                    if i == 0:
                        ##############
                        ############## These are the parameters of our first datafile.
                        ##############
                        
                        #Input image.
                        file_str = path+'/Concatenated Stacks_resized-1_every10.tif'
                        #User training.
                        #ROI_str = path+'/Concatenated Stacks_resized-1_every10_2.quantiROI'
                        #ROI_str = path+'/NBtraining'+str(21)+'.quantiROI'
                        #ROI_str = path+'/NBtraining21_withNeg.quantiROI'
                        ROI_str = path+'/NBtraining_first_and_last.quantiROI'
                        #or
                        #MDF=True
                        #The ground-truth. Indexed by time-point.
                        gt_time_pt = {}
                        gt_time_pt[0]  = path+'Concatenated Stacks_resized-1_every10.tif.mdf'
                        gt_time_pt[1]  = path+'Concatenated Stacks_resized-1_every10.tif2.mdf'
                        gt_time_pt[2]  = path+'Concatenated Stacks_resized-1_every10.tif3.mdf'
                        gt_time_pt[3]  = path+'Concatenated Stacks_resized-1_every10.tif4.mdf'
                        gt_time_pt[4]  = path+'Concatenated Stacks_resized-1_every10.tif5.mdf'
                        gt_time_pt[5]  = path+'Concatenated Stacks_resized-1_every10.tif6.mdf'
                        gt_time_pt[6]  = path+'Concatenated Stacks_resized-1_every10.tif7.mdf'
                        gt_time_pt[7]  = path+'Concatenated Stacks_resized-1_every10.tif8.mdf'
                        gt_time_pt[8]  = path+'Concatenated Stacks_resized-1_every10.tif9.mdf'
                        gt_time_pt[9]  = path+'Concatenated Stacks_resized-1_every10.tif10.mdf'
                        gt_time_pt[10] = path+'Concatenated Stacks_resized-1_every10.tif11.mdf'
                        gt_time_pt[11] = path+'Concatenated Stacks_resized-1_every10.tif12.mdf'
                        gt_time_pt[12] = path+'Concatenated Stacks_resized-1_every10.tif13.mdf'
                        gt_time_pt[13] = path+'Concatenated Stacks_resized-1_every10.tif14.mdf'
                        gt_time_pt[14] = path+'Concatenated Stacks_resized-1_every10.tif15.mdf'
                        
                        par_obj.file_array.append(file_str)
                        par_obj.gt_array.append(gt_time_pt)
                

            #Gets the dimensional information from the first file.
            #Assumes they are all the same. May not be true, but Martin you changed the data_structure :-).
            v2.import_data_fn(par_obj,[par_obj.file_array[0]])
            try:
                time_list = np.arange(0,par_obj.limit_time_pt)
                par_obj.total_time_pt = par_obj.limit_time_pt
            except:
                time_list = np.arange(0,par_obj.total_time_pt)
            
            par_obj.time_pt_list = np.arange(0,par_obj.total_time_pt)
            par_obj.max_file = nstr.__len__()
            
            
            
            par_obj.initiate_data_store()
            par_obj.height = par_obj.ori_height
            par_obj.width = par_obj.ori_width
            fileno=0
            ########### 
            ###########This is where we start our peformance calculation.
            ###########

            #This is the container for the ground-truths points that we load in.
            par_obj.TP[training][fileno]={}
            par_obj.FP[training][fileno]={}
            par_obj.FN[training][fileno]={}
            par_obj.data_store['gt_pts']={}
            par_obj.data_store['gt_pts'][fileno] = {}
            for time_pt in par_obj.time_pt_list:

                par_obj.TP[training][fileno][time_pt]=[]
                par_obj.FP[training][fileno][time_pt]=[]
                par_obj.FN[training][fileno][time_pt]=[]
                for num_of_train,thr in enumerate(range(1,5000,25)):
                    par_obj.data_store['pts'][fileno][time_pt]=[]
                    par_obj.rel_thr = thr
                    #sample randomly in volume, without replacement
                    indices = numpy.random.choice(par_obj.height*par_obj.width*par_obj.max_zslices/2,size=thr,replace=False)*2
                    #covert index to coordinate
                    [cx,cy,cz]=numpy.unravel_index(indices, (par_obj.height,par_obj.width,par_obj.max_zslices))
                    pts_arr=[[x,y,z,1] for x,y,z in zip(cx,cy,cz)]
                    pts=[]
                    #remove indices within range of each other
                    pts.append(pts_arr[0])
                    for i in range(1,len(pts_arr)):
                        skip=False
                        [x0,y0,z0,W]=pts_arr[i]
                        for x1,y1,z1,W in pts:
                            dis = np.sqrt((y1-y0)**2 + (x1-x0)**2  +((z1-z0)/par_obj.z_calibration)**2)
                            if dis < pixel_cut_off_3D:
                                skip=True
                                break
                        if skip==False:
                            pts.append(pts_arr[i])
                            
                    pts_arr=pts
                    par_obj.data_store['pts'][fileno][time_pt]=pts_arr

                    gt_pts = read_ground_truth(par_obj.gt_array[fileno][time_pt],time_pt)
                    par_obj.data_store['gt_pts'][fileno][time_pt] = gt_pts

                    ###########
                    ########### Plot all the points.
                    ###########
                    #for y0,x0,z0 in par_obj.data_store['pts'][fileno][time_pt]:
                    #    plot(x0,y0,'ro')
                    #for y1,x1,z1 in par_obj.data_store['gt_pts'][fileno][time_pt]:
                    #    plot(x1,y1,'bo')
                    ###########
                    ###########
                    ###########

                    rel = []
                    gbl_min = []

                    #Next we want to for every predicted point calculate the distance to the nearest ground-truth point.
                    for y0,x0,z0,W in par_obj.data_store['pts'][fileno][time_pt]:
                        dist_arr = []
                        for y1,x1,z1 in par_obj.data_store['gt_pts'][fileno][time_pt]:
                            #Euclidean distance. #using z_calibration to convert to real 3D space and not voxels
                            dis = np.sqrt((y1-y0)**2 + (x1-x0)**2  +((z1-z0)/par_obj.z_calibration)**2)
                            dist_arr.append(dis)
                        #Nearest distance indice 
                        rel.append(np.argmin(dist_arr))
                        #Nearest distance distance.
                        gbl_min.append(np.min(dist_arr))

                    pred_to_keep = []
                    gt_to_keep = []
                    dist_in_3D_pixels = []


                    #Go through all of the relatives.
                    for i in set(rel):
                        #Find the indices where there a multiple nearest matches.
                        ind = np.where(np.array(rel) == i)[0]
                        #Finds the minimum index of the valid distances.
                        dist_ind = ind[np.argmin(np.array(gbl_min)[ind])]
                        min_dist = np.min(np.array(gbl_min)[ind])
                        #pts which are nearest.
                        pred_to_keep.append(dist_ind)
                        gt_to_keep.append(rel[dist_ind])
                        dist_in_3D_pixels.append(min_dist)


                    ########### This is where we compare the two lists.
                    ########### We can extract the x,y,z coordinates if we want more elaborate comparison than 3D Distance
                    ########### 
                    #Trim the 3-D distances down to the ones within a distances. Default: pixel_cut_off_3D
                    index_valid =  np.array(dist_in_3D_pixels) < pixel_cut_off_3D
                    #Take those indices which match this from the nearest neighbour short-list.
                    pred_to_keep_within_dist = np.array(pred_to_keep)[index_valid]
                    gt_to_keep_within_dist = np.array(gt_to_keep)[index_valid]
                    '''
                    figure(figsize=(16,4)) #uncomment to make multiple plots
                    #for y0,x0,z0,W in [par_obj.data_store['pts'][fileno][time_pt][i] for i in pred_to_keep]:
                    for y0,x0,z0,W in par_obj.data_store['pts'][fileno][time_pt]:
                        h=subplot(1,4,1)
                        h.axes.axis((0,par_obj.crop_x2,0,par_obj.crop_y2))
                        plot(x0,y0,'wo')
                        h=subplot(1,4,2)
                        h.axes.axis((0,par_obj.crop_x2,0,par_obj.max_zslices))
                        plot(x0,z0,'wo')

                    #for y1,x1,z1 in np.array(par_obj.data_store['gt_pts'][fileno][time_pt])[gt_to_keep]:
                    for y1,x1,z1 in par_obj.data_store['gt_pts'][fileno][time_pt]:
                        subplot(1,4,1)
                        plot(x1,y1,'ko')
                        subplot(1,4,2)
                        plot(x1,z1,'ko')
                    ###########
                    ###########
                    ###########

                    ########### This is for plotting the distances
                    ###########
                    ###########
                    #Plot those remaining. Can also
                    for y0,x0,z0,W in [par_obj.data_store['pts'][fileno][time_pt][i] for i in pred_to_keep_within_dist]:
                        h=subplot(1,4,3)
                        h.axes.axis((0,par_obj.crop_x2,0,par_obj.crop_y2))
                        plot(x0,y0,'ro')
                        h=subplot(1,4,4)
                        h.axes.axis((0,par_obj.crop_x2,0,par_obj.max_zslices))
                        plot(x0,z0,'ro')
                    for y1,x1,z1 in np.array(par_obj.data_store['gt_pts'][fileno][time_pt])[gt_to_keep_within_dist]:
                        subplot(1,4,3)
                        plot(x1,y1,'go')
                        subplot(1,4,4)
                        plot(x1,z1,'go')
                    ###########
                    ###########
                    ###########'''

                    num_gt = float(par_obj.data_store['gt_pts'][fileno][time_pt].__len__())
                    num_hits = float(gt_to_keep_within_dist.__len__())
                    ###########
                    ########### These are the details we are interested in.
                    ###########

                    print '--------------------------------------------------------------------------------'
                    print 'filename',par_obj.file_array[fileno]
                    print 'time point',time_pt
                    print 'number of training dots', thr
                    predicted_Centres=float( len(par_obj.data_store['pts'][fileno][time_pt]))
                    print 'number of predicted centres', predicted_Centres
                    print 'ground_truths total', num_gt
                    print 'predicted points total', float(len(par_obj.data_store['pts'][fileno][time_pt]))
                    par_obj.TP[training][fileno][time_pt].append(num_hits)
                    print 'Valid centres (True positives)', par_obj.TP[training][fileno][time_pt]
                    par_obj.FP[training][fileno][time_pt].append(predicted_Centres-float(len(pred_to_keep_within_dist)))
                    print 'Invalid centres (False positives)',par_obj.FP[training][fileno][time_pt]
                    par_obj.FN[training][fileno][time_pt].append(float(num_gt)-float(len(pred_to_keep_within_dist)))
                    print 'Non-matched ground-truth points (False negatives)', par_obj.FN[training][fileno][time_pt]
                    ###########
                    ###########
                    ###########
                    #write output as we go along

                    output= [num_hits,
                            predicted_Centres-float(len(pred_to_keep_within_dist)),
                             float(num_gt)-float(len(pred_to_keep_within_dist)),
                             time_pt,
                             thr,
                             training,
                             fileno,
                            bc]
                    with open(csvPath+'outputData.csv', 'a') as csvfile:
                        spamwriter = csv.writer(csvfile,  dialect='excel')
                        spamwriter.writerow(output)

                    '''
                    csvPath = path+"/outputdata/";
                    output = '\tPath of data:\t',path,'\time to train\t',time_to_train,'\tnumber of training images.\t',num_of_train,'\tthe score\t',rf_score
                    with open(csvPath+'outputDataLastTrainFIN.csv', 'a') as csvfile:
                        spamwriter = csv.writer(csvfile,  dialect='excel')
                        spamwriter.writerow(output)
                    per_err =[]
                    for i,item in enumerate(test_index):
                        test= False
                        for c in train_index:
                            if item  == c:
                                test = True;
                        per_err.append((1-(abs(par_obj.gt_dense[item] - par_obj.final_prediction[item])/par_obj.gt_dense[item]))*100)
                        #output =  "\timage_id\t",item,"\ttime to process\t",time_taken_to_calc_feat[i],"\ttime to evaluate\t",time_taken_to_eval_trees[i],"\ttest\t",test,"\tgt_dense\t",par_obj.gt_dense[item],"\tfinal_predicion\t",par_obj.final_prediction[item],"\tper_err\t",per_err,"\tthe_score\t",par_obj.the_score[item]
                    output = "\toutput\t",np.average(per_err)
                    print output
                    with open(csvPath+'outputDataLastTrainFIN.csv', 'a') as csvfile:
                            spamwriter = csv.writer(csvfile,  dialect='excel')
                            spamwriter.writerow(output)"""
                    '''

[0] [0]
tiff resolution not recognised
--------------------------------------------------------------------------------
filename /Users/martin/Pictures/Live_mdf//Concatenated Stacks_resized-1_every10.tif
time point 0
number of training dots 1
number of predicted centres 1.0
ground_truths total 21.0
predicted points total 1.0
Valid centres (True positives) [0.0]
Invalid centres (False positives) [1.0]
Non-matched ground-truth points (False negatives) [21.0]
--------------------------------------------------------------------------------
filename /Users/martin/Pictures/Live_mdf//Concatenated Stacks_resized-1_every10.tif
time point 0
number of training dots 26
number of predicted centres 25.0
ground_truths total 21.0
predicted points total 25.0
Valid centres (True positives) [0.0, 0.0]
Invalid centres (False positives) [1.0, 25.0]
Non-matched ground-truth points (False negatives) [21.0, 21.0]
--------------------------------------------------------------------------------
filename /Users/

In [ ]:
pos_regions

In [ ]:
time_pt=1
figure()
false_positives=np.array(par_obj.FP[1][0][time_pt])
true_positives=np.array(par_obj.TP[1][0][time_pt])
false_negatives=np.array(par_obj.FN[1][0][time_pt])
plot(false_positives/(false_negatives+true_positives))
plot(true_positives/(false_negatives+true_positives))
plot(false_negatives/(false_negatives+true_positives))

In [3]:
col=['m','r','y','g','b','c']

training=par_obj.FP.keys()
training=training[0:]#[1:21:2]
tpts=range(1,14)
figure(figsize=(6,6))
color=np.array((0,0,1))
for t in training:
    false_positives=0
    true_positives=0
    false_negatives=0
    for tpt in tpts[1:]:

        false_positives+=np.array(par_obj.FP[t][0][tpt])
        true_positives+=np.array(par_obj.TP[t][0][tpt])
        false_negatives+=np.array(par_obj.FN[t][0][tpt])
        Recall=true_positives/(true_positives+false_negatives)
        Precision = true_positives/(true_positives+false_positives)
        #plot(Precision)

    plot(Recall,Precision,'x-',color =tuple(color))
    axis((0,1,0,1))
    color=color-np.array((0,0,0.075))
    #color[np.where(color>0.8)]=0
'''for t in training:
    for tpt in tpts:
        count+=1
        false_positives=np.array(par_obj.FP[t][0][tpt])
        true_positives=np.array(par_obj.TP[t][0][tpt])
        false_negatives=np.array(par_obj.FN[t][0][tpt])
        Recall=true_positives/(true_positives+false_negatives)
        Precision = true_positives/(true_positives+false_positives)
        #plot(Precision)
        plot(Recall,Precision,col[count%6]+'x-')
        axis((0,1,0,1))'''
plot(Recall,Precision,color =(1,0,0))
plot(Recall[0:1],Precision[0:1],'o')

KeyError: 0

In [16]:
pts_arr

[[446, 381, 18, 1]]

In [37]:
par_obj.saved_ROI

[(0, 49, 59, 108, 363, 0, 0), (13, 396, 252, 56, 63, 0, 0)]

In [1]:
print 5

5


In [48]:
color[ind]=color[ind]+0.2

In [49]:
color[ind]

0

In [8]:
par_obj.ori_width

512

{}

In [5]:
par_obj.tiff_file

In [7]:
par_obj.height

0

In [1]:
par_obj.limit_ratio_size

NameError: name 'par_obj' is not defined

In [10]:
par_obj.tiff_file.pages[0]

In [29]:
par_obj.saved_ROI

[(0, 49, 59, 108, 363, 0, 0),
 (0, 176, 231, 59, 57, 0, 0),
 (0, 201, 169, 67, 54, 0, 0),
 (0, 262, 242, 46, 57, 0, 0),
 (2, 232, 236, 44, 67, 0, 0),
 (2, 234, 110, 72, 72, 0, 0),
 (2, 336, 170, 72, 78, 0, 0),
 (5, 180, 229, 64, 56, 0, 0),
 (5, 205, 171, 49, 54, 0, 0),
 (7, 268, 190, 58, 47, 0, 0),
 (7, 329, 277, 38, 61, 0, 0),
 (10, 344, 159, 63, 60, 0, 0),
 (10, 250, 322, 82, 70, 0, 0),
 (13, 396, 252, 56, 63, 0, 0),
 (14, 254, 318, 68, 67, 0, 0),
 (19, 350, 91, 80, 95, 0, 0),
 (2, 179, 232, 59, 53, 0, 0),
 (5, 301, 152, 43, 52, 0, 0),
 (7, 288, 291, 39, 53, 0, 0),
 (9, 366, 261, 46, 52, 0, 0),
 (12, 117, 353, 64, 57, 0, 0),
 (12, 367, 271, 43, 49, 0, 0),
 (17, 280, 55, 67, 71, 0, 0)]

In [33]:
idx

13

In [3]:
par_obj.time_pt_list

array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9])

In [5]:
par_obj.height

512

In [9]:
par_obj.max_zslices

20

In [11]:
par_obj.height*par_obj.width*par_obj.max_zslices

5242880

In [12]:
thr

[1,
 6,
 11,
 16,
 21,
 26,
 31,
 36,
 41,
 46,
 51,
 56,
 61,
 66,
 71,
 76,
 81,
 86,
 91,
 96,
 101,
 106,
 111,
 116,
 121,
 126,
 131,
 136,
 141,
 146,
 151,
 156,
 161,
 166,
 171,
 176,
 181,
 186,
 191,
 196]

In [18]:
pts_arr

[[461, 132, 12, 1],
 [504, 332, 1, 1],
 [328, 47, 10, 1],
 [423, 68, 0, 1],
 [444, 116, 19, 1],
 [43, 367, 16, 1],
 [393, 28, 4, 1],
 [140, 319, 10, 1],
 [154, 361, 10, 1],
 [369, 248, 5, 1],
 [81, 229, 14, 1],
 [158, 159, 1, 1],
 [69, 340, 17, 1],
 [64, 430, 1, 1],
 [406, 456, 6, 1],
 [312, 71, 2, 1],
 [494, 328, 19, 1],
 [221, 99, 18, 1],
 [169, 386, 5, 1],
 [235, 42, 0, 1],
 [344, 460, 13, 1],
 [351, 149, 6, 1],
 [162, 273, 9, 1],
 [119, 46, 2, 1],
 [319, 23, 8, 1],
 [353, 95, 17, 1],
 [460, 313, 14, 1],
 [129, 182, 8, 1],
 [0, 237, 6, 1],
 [413, 300, 11, 1],
 [144, 360, 11, 1],
 [104, 89, 10, 1],
 [271, 158, 8, 1],
 [27, 421, 12, 1],
 [274, 194, 2, 1],
 [100, 38, 16, 1],
 [221, 380, 0, 1],
 [193, 390, 7, 1],
 [133, 326, 19, 1],
 [353, 189, 13, 1],
 [237, 271, 11, 1],
 [172, 1, 3, 1],
 [490, 310, 17, 1],
 [418, 462, 15, 1],
 [57, 356, 0, 1],
 [10, 271, 12, 1],
 [417, 344, 17, 1],
 [180, 66, 1, 1],
 [133, 367, 13, 1],
 [9, 249, 7, 1],
 [492, 481, 19, 1],
 [201, 471, 16, 1],
 [293, 13

In [19]:
gt_pts

[[155.35674931129478, 327.43388429752065, 13.0],
 [151.12534435261708, 294.9931129476584, 13.0],
 [252.67906336088154, 388.7892561983471, 14.0],
 [192.73415977961432, 382.44214876033055, 14.0],
 [279.4779614325069, 428.9876033057851, 18.0],
 [307.6873278236915, 403.599173553719, 18.0],
 [307.6873278236915, 403.599173553719, 18.0],
 [323.90771349862257, 326.0234159779614, 10.0],
 [304.866391184573, 330.9600550964187, 9.0],
 [332.37052341597797, 297.10881542699724, 15.0],
 [361.28512396694214, 213.18595041322314, 11.0],
 [378.2107438016529, 159.58815426997245, 14.0],
 [304.1611570247934, 239.27961432506888, 6.0],
 [266.78374655647383, 266.07851239669424, 8.0],
 [266.07851239669424, 318.9710743801653, 7.0],
 [204.01790633608815, 310.5082644628099, 7.0],
 [201.9022038567493, 254.08953168044079, 3.0],
 [233.63774104683196, 237.16391184573004, 8.0],
 [238.57438016528926, 213.89118457300276, 4.0]]

In [22]:
ind

array([ 11,  17,  19,  23,  31,  32,  34,  35,  41,  47,  62,  66,  68,
        69,  74,  76,  93, 107, 114, 118, 119, 132, 133, 159, 172, 174,
       176, 178, 186])

In [23]:
dist_in_3D_pixels

[17.548793512772267,
 13.93620388916017,
 7.9143054545672005,
 14.581897096721898,
 15.165188546015878,
 39.127805841774183,
 5.2342884571684607,
 21.537917976631054,
 25.574360617284885,
 9.7342125019183641,
 4.3682257532416555,
 15.970347406883436,
 28.265380733031222,
 8.2004358676123914,
 22.228262550960281,
 30.043466665415462]

In [24]:
pred_to_keep_within_dist

array([ 65,  37, 153, 166,  81,  91, 102])

In [25]:
print '--------------------------------------------------------------------------------'
print 'filename',par_obj.file_array[fileno]
print 'time point',time_pt
print 'number of training dots', thr
predicted_Centres=float( len(par_obj.data_store['pts'][fileno][time_pt]))
print 'number of predicted centres', predicted_Centres
print 'ground_truths total', num_gt
print 'predicted points total', float(len(par_obj.data_store['pts'][fileno][time_pt]))
par_obj.TP[training][fileno][time_pt].append(num_hits)
print 'Valid centres (True positives)', par_obj.TP[training][fileno][time_pt]
par_obj.FP[training][fileno][time_pt].append(predicted_Centres-float(len(pred_to_keep_within_dist)))
print 'Invalid centres (False positives)',par_obj.FP[training][fileno][time_pt]
par_obj.FN[training][fileno][time_pt].append(float(num_gt)-float(len(pred_to_keep_within_dist)))
print 'Non-matched ground-truth points (False negatives)', par_obj.FN[training][fileno][time_pt]


--------------------------------------------------------------------------------
filename /Users/martin/Pictures/Live_mdf//Concatenated Stacks_resized-1_every10.tif
time point 14
number of training dots 196
number of predicted centres 196.0
ground_truths total 21.0
predicted points total 196.0
Valid centres (True positives) [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
Invalid centres (False positives) [1.0, 6.0, 10.0, 15.0, 21.0, 25.0, 29.0, 33.0, 38.0, 43.0, 47.0, 55.0, 59.0, 64.0, 69.0, 72.0, 77.0, 81.0, 89.0, 89.0, 98.0, 103.0, 109.0, 111.0, 115.0, 123.0, 128.0, 130.0, 134.0, 141.0, 144.0, 150.0, 155.0, 161.0, 163.0, 170.0, 175.0, 180.0, 181.0, 189.0, 189.0]
Non-matched ground-truth points (False negatives) [21.0, 21.0, 20.0, 20.0, 21.0, 20.0, 19.0, 18.0, 18.0, 18.0, 17.0, 20.0, 19.0, 19.0, 19.0, 17.0, 17.0, 16.0, 19.0, 14.0